In [91]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

import warnings
warnings.filterwarnings("ignore")

read_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_region = 'Region'
col_year = 'Year'
col_gdp = 'GDP'

In [92]:
df = pd.read_csv(read_path + 'GoldDataframe.csv', index_col = [col_country, col_region, col_year])
df

AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  \
Country     Region             Year                                  
Afghanistan South Asia         2000              NaN   3342.034168   
                               2001              NaN   3598.470576   
                               2002              NaN   4141.523943   
                               2003              NaN   4729.042179   
                               2004              NaN   5388.482107   
...                                              ...           ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073   
                               2017              NaN  22040.902301   
                               2018              NaN  24311.560545   
                               2019              NaN  21935.075306   
                               2020              NaN  21786.743637   

                                     % Soldiers  Employment in industry  \
Country     Region             Year                                       
Afghanistan South Asia         2000    7.887961                    9.48   
                               2001    5.020511                    8.98   
                               2002    2.153062                    9.99   
                               2003    2.208290                   10.35   
                               2004    0.435599                   10.61   
...                                         ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016    0.759750                    7.05   
                               2017    0.750720                    6.90   
                               2018    0.738210                    6.75   
                               2019    0.738210                    6.57   
                               2020    0.738210                    6.57   

                                     Employment in services  Birth Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000               24.680000      48.021   
                               2001               24.719999      47.505   
                               2002               25.590000      46.901   
                               2003               25.950001      46.231   
                               2004               26.120001      45.507   
...                                                     ...         ...   
Zimbabwe    Sub-Saharan Africa 2016               26.070000      32.864   
                               2017               26.629999      31.732   
                               2018               27.230000      30.676   
                               2019               27.240000      29.747   
                               2020               27.240000      28.977   

                                     Cost business start-up  Death Rate  ...  \
Country     Region             Year                                      ...   
Afghanistan 

In [103]:
PVALUE_VAR = 0.05 #Preguntar por %

In [110]:
country_list = list(set(df.index.get_level_values(col_country)))
region_list = list(set(df.index.get_level_values(col_region)))

country_list.sort()
region_list.sort()

shifts = range(0, 20)  # GENERALIZAR PARA CUALQUIER RANGO DE AÑOS

max_list = []
shift_list = []

faulty_list = []

for country in country_list:
    try: 
    #if True:
        print(country)
        corr_list = []
        pval_list = []
        
        
        #Get the DataFrame for a given country.
        country_df = df.loc[df.index.get_level_values(col_country) == country]
        min_year = country_df.index.get_level_values(col_year).min()
        max_year = country_df.index.get_level_values(col_year).max()

        gdp_series = country_df.loc[:, col_gdp]
        ind_df = country_df.drop(col_gdp, axis = 1).dropna(axis = 1)

        #corr_df = ind_df.corrwith(gdp_series, method = 'spearman')
        #display(corr_df)

        for shift in shifts:
            shifted_min = min_year + shift
            shifted_max = max_year - shift

            shifted_gdp = gdp_series.loc[gdp_series.index.get_level_values(col_year) >= shifted_min].reset_index(drop = True)
            shifted_ind = ind_df.loc[ind_df.index.get_level_values(col_year) <= shifted_max].reset_index(drop = True)

            country_corr = shifted_ind.corrwith(shifted_gdp, method = 'spearman')
            pval = shifted_ind.corrwith(shifted_gdp, method = lambda x, y: spearmanr(x, y)[1])
            p = pval.map(lambda x: 1 if x < PVALUE_VAR else np.NaN)
            country_corr = country_corr * p

            country_corr = country_corr.map(lambda x: np.NaN if x == 1.0 or x == -1.0 else x)

            #display(country_corr)
            #display(country_corr_b)
            #display(pval)

            country_corr.name = shift
            corr_list.append(country_corr)
            pval_list.append(pval)

        corr_df = pd.concat(corr_list, axis = 1).transpose().sort_index()
        pval_df = pd.concat(pval_list, axis = 1).transpose().sort_index()

        corr_df.index = pd.MultiIndex.from_tuples((country, sh) for sh, _ in corr_df.iterrows())
        corr_df.index.names = [col_country, 'Shift']

        val = corr_df.apply(lambda x: max(x.min(), x.max(), key=abs))
        val.name = country

        max_row = pd.DataFrame(val)
        #max_shift = pd.Series(val).map(lambda x: corr_df.loc[corr_df[pd.Series(val).loc[pd.Series(val) == x].index[0]] == x].index.get_level_values('Shift').item())
        max_shift = pd.DataFrame(val).applymap(lambda x: corr_df.loc[corr_df[pd.Series(val).loc[pd.Series(val) == x].index[0]] == x].index.get_level_values('Shift').item())


        #display(corr_df)
        #display(pval_df)

        max_list.append(max_row)
        shift_list.append(max_shift)
        #break
    except Exception: 
        faulty_list.append(country)
        

max_df = pd.concat(max_list, axis = 1).transpose().sort_index()
shift_df = pd.concat(shift_list, axis = 1).transpose().sort_index()

print(faulty_list)

display(max_df)
display(shift_df)

Afghanistan
Albania
Algeria
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bosnia and Herzegovina
Botswana
Brazil
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Costa Rica
Croatia
Cuba
Cyprus
Denmark
Djibouti
Dominican Republic
Ecuador
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
France
French Polynesia
Gabon
Georgia
Germany
Ghana
Greece
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kuwait
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Madagascar
Malawi
Malaysia
Maldives
Mali
Malta
Mauritania
Mauritius
Mexico
Mongolia
Montenegro
Morocco
Mozambique
Myanmar
Namibia
Nepal
Netherlands
New Caledonia
New Zealand
Nicaragua
Niger
Nigeria
North Macedonia
Norway
Oman
Pakistan
Panama
Papua New Guinea
Paraguay
Peru
Phi

,AgriShareGDP,% Soldiers,Employment in industry,Employment in services,Birth Rate,Cost business start-up,Death Rate,Employment in agriculture,% Female Employment,% Male Employment,...,Freedom of Expression,% Healthcare Investment,Population,% Education Expenditure,Homicides,Tertiary School Gender Parity,CreditToAgriFishForest,Researchers in R&D,R&D expenditure %GDP,EmploymentRural
Afghanistan,-0.879278,-0.828571,0.910685,0.911984,-0.914286,-0.754567,-0.914286,-0.910685,0.900000,-0.705653,...,0.796308,0.640861,0.914286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,-0.811679,-0.828070,0.828571,0.901961,-0.878526,-0.824438,0.885714,-0.875000,-0.833333,-0.821429,...,0.767193,0.885714,-0.888312,-0.582418,-0.671930,1.000000,NaN,NaN,NaN,NaN
Algeria,-0.763182,0.800000,0.853152,0.753491,0.874026,-0.799816,-0.811765,-0.750487,0.814286,0.776471,...,0.744103,0.629927,0.750649,NaN,NaN,0.738578,NaN,NaN,NaN,NaN
Argentina,0.697041,0.942857,-0.785714,-0.803081,0.942857,0.512360,0.942857,-0.900000,0.838235,0.891176,...,-0.705732,0.942857,-0.942857,1.000000,0.898645,0.766483,-0.736364,-0.900000,0.910526,NaN
Australia,-0.757910,-0.900000,0.857143,0.885714,0.768763,-0.906795,-0.695878,-0.829159,0.824561,-0.660606,...,-0.828079,-0.733333,0.816883,NaN,-0.792779,NaN,-0.660606,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uruguay,0.706152,-0.934981,-0.573684,0.837209,-0.929825,-0.951972,-0.854545,-0.835088,0.915377,0.858824,...,0.921206,0.833388,0.902597,NaN,0.764630,NaN,0.900000,NaN,0.827273,NaN
Uzbekistan,-0.875581,-0.733333,-0.916667,0.960784,0.828571,0.913223,-0.874788,-0.946336,0.790909,0.890110,...,0.525872,0.925625,0.850649,NaN,NaN,0.942857,NaN,-0.794805,0.942857,NaN
Vanuatu,-0.851938,NaN,0.967197,0.793939,-0.971429,0.862489,-0.958921,-0.939916,0.900000,0.857143,...,0.712667,0.942857,0.971429,NaN,NaN,NaN,-0.629784,NaN,NaN,NaN
Zambia,-0.815200,-0.850456,0.863548,0.697629,-0.837662,-0.731561,-0.837662,-0.723612,-0.642857,0.623529,...,-0.925820,-0.876471,0.837662,0.828571,NaN,NaN,0.728456,NaN,NaN,NaN


,AgriShareGDP,% Soldiers,Employment in industry,Employment in services,Birth Rate,Cost business start-up,Death Rate,Employment in agriculture,% Female Employment,% Male Employment,...,Freedom of Expression,% Healthcare Investment,Population,% Education Expenditure,Homicides,Tertiary School Gender Parity,CreditToAgriFishForest,Researchers in R&D,R&D expenditure %GDP,EmploymentRural
Afghanistan,1.0,15.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0,0.0,...,2.0,9.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,15.0,2.0,15.0,4.0,1.0,13.0,0.0,4.0,13.0,14.0,...,14.0,15.0,0.0,8.0,2.0,16.0,NaN,NaN,NaN,NaN
Algeria,6.0,12.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,5.0,...,6.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Argentina,10.0,15.0,13.0,7.0,15.0,4.0,15.0,16.0,4.0,5.0,...,0.0,15.0,15.0,16.0,15.0,0.0,10.0,16.0,2.0,NaN
Australia,5.0,16.0,14.0,15.0,4.0,0.0,8.0,0.0,3.0,11.0,...,15.0,11.0,0.0,NaN,1.0,NaN,11.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uruguay,10.0,0.0,2.0,2.0,2.0,2.0,10.0,2.0,3.0,5.0,...,7.0,0.0,0.0,NaN,0.0,NaN,16.0,NaN,10.0,NaN
Uzbekistan,3.0,12.0,12.0,3.0,15.0,13.0,3.0,3.0,10.0,8.0,...,0.0,0.0,0.0,NaN,NaN,15.0,NaN,0.0,15.0,NaN
Vanuatu,10.0,NaN,0.0,11.0,1.0,13.0,4.0,0.0,16.0,14.0,...,2.0,15.0,1.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN
Zambia,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,5.0,...,15.0,5.0,0.0,15.0,NaN,NaN,0.0,NaN,NaN,NaN


In [4]:
df1 = pd.DataFrame({1: [9, 3, 8], 2: [1, 5, 5], 3: [2, 2, 1]})
df1

df2 = pd.DataFrame({'t': ['a', 'a', 'b'], 1: [100, 388, 300], 2: [400, 124, 600], 3: [700, 977, 900]})
df2

df1.corrwith(df2)

display(df2)
dfs = df2.copy().loc[:, 1]
display(dfs)

df1.corrwith(dfs)
df2.groupby(['t']).sum()


,t,1,2,3
0,a,100,400,700
1,a,388,124,977
2,b,300,600,900


0    100
1    388
2    300
Name: 1, dtype: int64

,1,2,3
t,,,
a,488,524,1677
b,300,600,900
